# Floquet Cavity Cooling - GPU Training

This notebook runs training on Colab GPU.

**IMPORTANT**: After running cell 2, restart the runtime (Runtime > Restart runtime) before continuing!

In [1]:
# Cell 1: Install dependencies
!pip install -q jax[cuda12] jaxlib flax optax diffrax gymnasium stable-baselines3 matplotlib

In [2]:
# Cell 2: Clone repo and checkout dev branch
# After this cell, RESTART RUNTIME before continuing!
import os
if not os.path.exists('/content/abouie_proj'):
    !git clone https://github.com/sattary/abouie_proj.git /content/abouie_proj
else:
    %cd /content/abouie_proj
    !git fetch origin dev
    !git checkout dev
    !git pull origin dev
    %cd /content

print("\n" + "="*50)
print("NOW RESTART RUNTIME: Runtime > Restart runtime")
print("Then run cells starting from Cell 3")
print("="*50)

/content/abouie_proj
From https://github.com/sattary/abouie_proj
 * branch            dev        -> FETCH_HEAD
Already on 'dev'
Your branch is up to date with 'origin/dev'.
From https://github.com/sattary/abouie_proj
 * branch            dev        -> FETCH_HEAD
Already up to date.
/content

NOW RESTART RUNTIME: Runtime > Restart runtime
Then run cells starting from Cell 3


In [3]:
# Cell 3: Setup paths (run AFTER restart)
import sys
import os
os.chdir('/content/abouie_proj')
sys.path.insert(0, '/content/abouie_proj')

# Verify files exist
!ls src/
!cat src/physics/__init__.py | head -20

analysis  floquet      optimization  __pycache__  validation
baseline  __init__.py  physics	     rl
"""
Physics engine for cavity-qubit cooling.

Core modules:
- operators: Static quantum operators
- hamiltonian: Time-dependent Hamiltonian with dual control
- lindblad: Full Lindblad master equation
- initial_states: Quantum state preparation
"""

from .operators import (
    SystemParams,
    Operators,
    build_operators,
    thermal_occupation,
)

from .hamiltonian import (
    build_hamiltonian_func,
    build_hamiltonian_from_arrays,


In [4]:
# Cell 4: Verify GPU
import jax
devices = jax.devices()
print(f"JAX devices: {devices}")
gpu_available = any('cuda' in str(d).lower() or 'gpu' in str(d).lower() for d in devices)
print(f"GPU available: {gpu_available}")
assert gpu_available, "GPU not detected! Enable GPU in Runtime > Change runtime type"

JAX devices: [CudaDevice(id=0)]
GPU available: True


In [5]:
# Cell 5: Import modules
from src.physics import SystemParams, build_operators, thermal_cavity_ground_qubits
from src.floquet import create_constant_cycle, find_floquet_steady_state
from src.baseline import compute_stochastic_limit, StochasticParams
from src.optimization import run_grape_optimization, GRAPEConfig
from src.rl import FloquetCoolingEnv, train_sac
print("All imports OK!")

All imports OK!


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## 1. Quick Physics Test

In [6]:
# Test physics engine
params = SystemParams(kappa=0.05, gamma1=0.01, T_bath=0.5, T_atom=0.05)
ops = build_operators(params)
rho = thermal_cavity_ground_qubits(params)

import jax.numpy as jnp
n_init = float(jnp.real(jnp.trace(ops.n_cav @ rho)))
print(f"Initial cavity occupation: {n_init:.4f}")

Initial cavity occupation: 1.1243


## 2. GRAPE Optimization (GPU-accelerated)

In [7]:
# Run GRAPE optimization
config = GRAPEConfig(
    n_steps=20,
    T_cycle=0.5,
    n_cycles_eval=100,
    learning_rate=0.02,
    n_iterations=200,
    g_max=1.5,
    delta_max=0.3,
)

optimal_cycle, history = run_grape_optimization(params, config)

print(f"\nFinal <n>: {history[-1]:.4f}")

GRAPE Optimization


/usr/local/lib/python3.12/dist-packages/jax/_src/lax/lax.py:5473: ComplexWarning: Casting complex values to real discards the imaginary part
  x_bar = _convert_element_type(x_bar, x.aval.dtype, x.aval.weak_type)


Iter 20: <n> = 1.0262
Iter 40: <n> = 1.0260
Iter 60: <n> = 1.0257
Iter 80: <n> = 1.0255
Iter 100: <n> = 1.0252
Iter 120: <n> = 1.0249
Iter 140: <n> = 1.0246
Iter 160: <n> = 1.0244
Iter 180: <n> = 1.0242


KeyboardInterrupt: 

In [ ]:
# Plot GRAPE results
import matplotlib.pyplot as plt
import numpy as np

stoch = StochasticParams(
    omega_c=5.0, omega_a=5.0, kappa=0.05,
    T_bath=0.5, T_atom=0.05, lambda_ex=5.0,
    g=0.5, tau=0.05, R=5.0, chi=2.0,
)
n_stoch, _ = compute_stochastic_limit(stoch)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].plot(history)
axes[0].axhline(n_stoch, color='r', linestyle='--', label='Stochastic')
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('<n>')
axes[0].legend()

t = np.linspace(0, config.T_cycle, config.n_steps)
axes[1].step(t, optimal_cycle.g_sequence, 'g-', where='post')
axes[1].set_xlabel('Time (ns)')
axes[1].set_ylabel('g(t)')

axes[2].step(t, optimal_cycle.delta_sequence, 'purple', where='post')
axes[2].set_xlabel('Time (ns)')
axes[2].set_ylabel('delta(t)')

plt.tight_layout()
plt.show()

print(f"Stochastic limit: {n_stoch:.4f}")
print(f"Improvement: {(n_stoch - history[-1])/n_stoch*100:.1f}%")

## 3. SAC Training (longer run)

In [ ]:
# Train SAC agent (adjust timesteps as needed)
model, callback, env = train_sac(
    total_timesteps=50000,
    n_steps_per_cycle=20,
    n_cycles_per_episode=50,
)

In [ ]:
# Evaluate SAC results
from src.rl import evaluate_trained_agent

results, g_seq, delta_seq = evaluate_trained_agent(model, env, n_eval_episodes=5)

best_n = min(r['n_cav'] for r in results)
print(f"Best n_cav: {best_n:.4f} (target: {n_stoch:.4f})")